<a href="https://colab.research.google.com/github/smuratsirin/Retail_Market_Analysis/blob/main/Retail_Model_v0105.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is Retail Market mode and it is developed as
Generator Phase:

1.   Generator Phase

* Input: Projected demand.
* Process: Unit dispatch based on the projected demand.
* Output: Hourly price and emission data.

2.   Retail Phase

* Input: Hourly price and emission data from generators.
* Process: Calculation of prices to households, incorporating a profit margin.
* Output: Price information inclusive of profit margin.

3. Household Decision Phase:

* Input: Price and emission information from retail companies.
* Process: Households evaluate the information against their decision function to determine whether to switch their retailer.
* Output: Decision to stay with the current retailer or switch.
* Feedback: Information about household decisions is sent back to retail companies.

4. Retail Adjustment Phase:

* Input: Feedback from households on their decisions.
* Process: Retailers adjust profit margins based on household decisions to optimize future pricing strategies.
* Output: Adjusted profit margins.
* Implementation: Revised prices are prepared for the next cycle.



DEFINING FUNCTIONS

In [1]:
import os
os.getcwd()

'/content'

In [1]:
## This function generates random costs values for generators
def randomize_costs_and_emissions(df, columns, seed=42):
    # Seed for reproducibility
    np.random.seed(seed)

    # Create a copy of the dataframe to avoid modifying the original data
    modified_df = df.copy()

    # Apply randomization within ±20% for specified columns
    for column in columns:
        # Generate random factors between 0.8 (80%) and 1.2 (120%)
        random_factors = np.random.uniform(0.8, 1.2, size=df.shape[0])
        modified_df[column] = df[column] * random_factors

    # Apply additional division for the Emissions column
    modified_df['Emissions'] = modified_df['Emissions'] / 1000

    # Convert all column names to lowercase
    modified_df.columns = [col.lower() for col in modified_df.columns]

    return modified_df

In [2]:
import pandas as pd
import numpy as np
# Adjust assign_random_availability to include user-defined percentage
def generator_availability(power_plants, availability_percentage, seed=None):
    if seed is not None:
        np.random.seed(seed)  # for reproducibility
    # Determine the number of plants to be available based on the percentage
    num_available = int(len(power_plants) * availability_percentage / 100)
    # Create an array with the specified number of 1s (available) and the rest 0s (unavailable)
    availability = np.array([1] * num_available + [0] * (len(power_plants) - num_available))
    np.random.shuffle(availability)
    power_plants['availability'] = availability

In [4]:
# This function defines the generation_function
import pandas as pd
import numpy as np

def generation_function(peak_demand, demand_profile_24h, renewable_capacity, renewable_costs, solar_capacity_factor_24h, wind_capacity_factor_24h,  voll,  pricing, fossil_power_plants):
    """
    Estimates hourly generation costs, marginal prices, unserved demand, emissions, and calculates total
    average costs and emissions over a 24-hour period based on pricing strategy selected (average or marginal).

    Parameters:
    - peak_demand (float): Estimated peak demand for the period.
    - demand_profile_24h (list): Coefficients representing hourly demand profiles.
    - capacity (tuple): Capacities for solar, wind, and storage (in that order).
    - solar_capacity_factor_24h (list), wind_capacity_factor_24h (list): Hourly capacity factors for solar and wind.
    - costs (tuple): Marginal costs for solar, wind, and storage technologies.
    - voll (float): Value of Lost Load - cost of unserved electricity demand per unit.
    - pricing (str): Pricing strategy to use for the output, "average" or "marginal".
    - fossil_power_plants: This ius the dataset that includes the fixed, variable and emission values for fossil fuel technologies

    Returns:
    - tuple: Depending on the pricing strategy, returns lists of hourly generation, unserved demand,
             and either marginal prices and emissions or average prices and emissions.


    """

    peak_demand = float(peak_demand)

    demand_profile_24h = [float(x) for x in demand_profile_24h.split(',')] if isinstance(demand_profile_24h, str) else demand_profile_24h
    solar_capacity_factor_24h = [float(x) for x in solar_capacity_factor_24h.split(',')] if isinstance(solar_capacity_factor_24h, str) else solar_capacity_factor_24h
    wind_capacity_factor_24h = [float(x) for x in wind_capacity_factor_24h.split(',')] if isinstance(wind_capacity_factor_24h, str) else wind_capacity_factor_24h

    # Load oil and gas power plant data and assign availability based on the given percentage
    power_plants = fossil_power_plants.copy()

    # Select available power plants and sort by variable cost
    available_plants = power_plants[power_plants['availability'] == 1]
    available_plants = available_plants.sort_values(by='variable_costs')

    ## Add renewable capaciies and costs

    renewable_capacity = renewable_capacity.split(',')
    solar_capacity, wind_capacity, storage_capacity = map(float, renewable_capacity)

    renewable_costs = renewable_costs.split(',')
    solar_cost, wind_cost, storage_cost = map(float, renewable_costs)


    hourly_generation = []
    hourly_marginal_price = []
    hourly_unserved_demand = []
    hourly_emissions = []
    total_cost = 0
    total_emissions = 0
    storage_level = 0
    ## This availability is the technical availability of the power plant, it can use this ratio of its nameplate capacity
    availability_ratio = 0.9

    for hour in range(24):
        solar_factor = solar_capacity_factor_24h[hour]
        wind_factor = wind_capacity_factor_24h[hour]
        demand = peak_demand * demand_profile_24h[hour]
        solar_generation = solar_capacity * solar_factor
        wind_generation = wind_capacity * wind_factor
        renewable_generation = solar_generation + wind_generation

        emissions_for_hour = 0
        hourly_total_emissions = 0
        unserved_demand = 0

        if renewable_generation >= demand:
            excess_generation = renewable_generation - demand
            storage_space_available = storage_capacity - storage_level
            storage_level += min(excess_generation, storage_space_available)
            generation = demand
            marginal_cost = solar_cost if solar_generation >= wind_generation else wind_cost
            unserved_demand = 0
        else:
            generation = renewable_generation
            demand -= renewable_generation
            marginal_cost = solar_cost if solar_generation >= wind_generation else wind_cost

            if storage_level > 0 and demand > 0:
                from_storage = min(demand, storage_level)
                generation += from_storage
                storage_level -= from_storage
                demand -= from_storage
                marginal_cost = storage_cost

            for index, plant in available_plants.iterrows():
                if demand <= 0:
                    break
                plant_generation_available = plant['capacity'] * availability_ratio
                if plant_generation_available > demand:
                    additional_generation = demand
                else:
                    additional_generation = plant_generation_available

                hourly_total_emissions += plant['emissions'] * additional_generation
                total_cost += plant['fixed_costs'] + plant['variable_costs'] * additional_generation
                marginal_cost = plant['variable_costs']
                demand -= additional_generation
                generation += additional_generation
                hourly_average_emissions = hourly_total_emissions/generation

            if demand > 0:  # If demand is still not met after using all sources
                marginal_cost = voll  # Set the marginal cost to VOLL
                unserved_demand = demand


        hourly_emissions.append(hourly_average_emissions)
        total_emissions += hourly_total_emissions
        hourly_generation.append(generation)
        hourly_marginal_price.append(marginal_cost / 10)
        hourly_unserved_demand.append(unserved_demand)



    average_price = total_cost / (10 * sum(hourly_generation)) if sum(hourly_generation) > 0 else voll
    average_emissions = total_emissions / sum(hourly_generation) if sum(hourly_generation) > 0 else 0

    if pricing == "marginal":
        return hourly_marginal_price, hourly_emissions
    elif pricing == "average":
        return [average_price] * 24, [average_emissions] * 24
    else:
        raise ValueError("Invalid pricing option. Choose 'average' or 'marginal'.")


In [6]:
import numpy as np
import pandas as pd
from collections import Counter

def retail_offer_function(retailers, prices, emissions, time_periods):
    """
    Calculates retail offers for given retailers based on price and emission data over specified time periods.
    Determines benchmark prices and retailers.

    Args:
    retailers (list of tuples): List of (name, share) for each retailer.
    prices (list): List of prices over the time period.
    emissions (list): List of emissions over the same period.
    time_periods (list of tuples): List of (start, end) indices defining each time period.

    Returns:
    DataFrame: Contains offers, benchmark prices, and benchmark retailer information for each retailer.
    """

    def calculate_period_averages():
        """Calculate average price and emissions for each time period."""
        return [(np.mean(prices[start:end+1]), np.mean(emissions[start:end+1])) for start, end in time_periods]

    def generate_offers(period_averages):
        """Generate offers for each retailer based on their share and period averages."""
        offers = []
        for name, share in retailers:
            markup = 1 + (share / (1 + share)) if share > 0 else None
            offers.append([(p * markup, e) if markup else (None, None) for p, e in period_averages])
        return offers

    def find_benchmark_prices(offers):
        """Find the minimum price for each period among all offers."""
        return [min((offer[idx][0] for offer in offers if offer[idx][0] is not None), default=None) for idx in range(len(time_periods))]

    def find_benchmark_retailers(offers, benchmark_prices):
        """Determine the benchmark retailer for each period based on the benchmark price."""
        benchmark_retailer_names = []
        for idx, bp in enumerate(benchmark_prices):
            period_benchmark_retailers = [retailers[i][0] for i, offer in enumerate(offers) if offer[idx][0] is not None and offer[idx][0] == bp]
            benchmark_retailer_names.append(", ".join(period_benchmark_retailers) if period_benchmark_retailers else "No active benchmarks")
        return max(set(benchmark_retailer_names), key=benchmark_retailer_names.count)

    period_averages = calculate_period_averages()
    retailer_offers = generate_offers(period_averages)
    benchmark_prices = find_benchmark_prices(retailer_offers)
    benchmark_retailer = find_benchmark_retailers(retailer_offers, benchmark_prices)

    # Formatting the DataFrame
    data = {
        "Retail Firm": [retail[0] for retail in retailers],
        "Retail Price Offer": [[np.round(p, 2) if p is not None else "Out of business" for p, e in offers] for offers in retailer_offers],
        "Emission": [[np.round(e, 2) if p is not None else "Out of business" for p, e in offers] for offers in retailer_offers],
        "Benchmark Offer": [[np.round(bp, 2) if bp is not None else None for bp in benchmark_prices] for _ in retailers]  # Replicate benchmark prices across all rows
    }
    df = pd.DataFrame(data)
    df['Benchmark Retail Firm'] = benchmark_retailer  # Uniform benchmark retailer(s) across all rows

    return df

In [7]:
## Function to create initial household dataset for analysis; then no need to create one
## Do not forget to change values for energy demand and energy generation profiles

import numpy as np
import pandas as pd

def generate_weights(household_type):
    """
    Generates financial and carbon weights for a given household type.

    Parameters:
    - household_type (str): The type of the household (Small, Medium, Large).

    Returns:
    - tuple: (financial_weight, carbon_weight) where both sum to 1.
    """
    if household_type == 'Small':
        financial_weight = np.random.uniform(0.9, 1)
    elif household_type == 'Medium':
        financial_weight = np.random.uniform(0.7, 1)
    else:  # Large
        financial_weight = np.random.uniform(0.6, 1)

    carbon_weight = 1 - financial_weight
    return np.round(financial_weight, 2), np.round(carbon_weight, 2)

def create_household_dataset(N, seed=42, retailers_info=[("Retail A", 0.34), ("Retail B", 0.34), ("Retail C", 0.32)]):
    """
    Creates a dataset of households with detailed energy consumption profiles,
    solar PV capacity, and retailer assignments based on specified market shares.

    Parameters:
    - N (int): Number of households to generate.
    - seed (int): Seed for the random number generator for reproducibility.
    - retailers_info (list of tuples): A list where each tuple contains a retailer's name
      and their market share (e.g., [("Retail A", 0.45), ("Retail B", 0.3), ...]).

    Returns:
    - DataFrame: A pandas DataFrame containing the household dataset with their energy profiles,
      solar PV capacities, financial and carbon weights, initial retailer assignment, and default bias.
    """
    np.random.seed(seed)

    household_types = ['Small', 'Medium', 'Large']
    proportions = [0.4, 0.4, 0.2]
    base_consumption = {'Small': 1500, 'Medium': 3000, 'Large': 6000}
    std_deviation = {'Small': 200, 'Medium': 300, 'Large': 500}
    pv_capacity = {'Small': (0, 0), 'Medium': (0, 0), 'Large': (1000, 100)}

    retailers, market_shares = zip(*retailers_info)

    ## Correct thes econsumption patterns

    demand_profile_winter = np.array([0.111, 0.108, 0.112, 0.119, 0.126, 0.122])
    demand_profile_summer = np.array([0.213, 0.202, 0.217, 0.231, 0.223, 0.215])
    solar_pv_profile_winter = np.array([0.0,0.168,0.677,0.423,0.003,0.0])
    solar_pv_profile_summer = np.array([0.0,0.185,0.628,0.423,0.008,0.0])

    households = np.random.choice(household_types, size=N, p=proportions)
    retail_assignments = np.random.choice(retailers, size=N, p=market_shares)
    consumption_data = []

    for idx, household in enumerate(households):
        financial_weight, carbon_weight = generate_weights(household)
        base = np.random.normal(base_consumption[household], std_deviation[household])
        base = np.round(base, 2)  # Round base consumption

        mean_pv, std_pv = pv_capacity[household]
        solar_pv_capacity = np.round(np.random.normal(mean_pv, std_pv), 2) if std_pv > 0 else 0
        default_bias = np.round(np.random.uniform(0.05, 0.4), 2)  # Generate default bias

        consumption_winter = np.round(demand_profile_winter * base, 2)
        consumption_summer = np.round(demand_profile_summer * base, 2)
        solar_pv_production_winter = np.round(solar_pv_capacity * solar_pv_profile_winter, 2) if solar_pv_capacity else np.zeros(6)
        solar_pv_production_summer = np.round(solar_pv_capacity * solar_pv_profile_summer, 2) if solar_pv_capacity else np.zeros(6)

        preferred_consumption_winter = np.round([max(0, c - pv) for c, pv in zip(consumption_winter, solar_pv_production_winter)], 2)
        preferred_consumption_summer = np.round([max(0, c - pv) for c, pv in zip(consumption_summer, solar_pv_production_summer)], 2)

        consumption_data.append([
            idx + 1, household, base, solar_pv_capacity, financial_weight, carbon_weight,
            consumption_winter, consumption_summer, preferred_consumption_winter, preferred_consumption_summer,
            solar_pv_production_winter, solar_pv_production_summer, retail_assignments[idx], default_bias
        ])

    columns = [
        'Household ID', 'Household Type', 'Base Consumption', 'Solar PV Capacity', 'Financial Weight', 'Carbon Weight',
        'Winter Gross', 'Summer Gross', 'Winter Preferred', 'Summer Preferred',
        'Solar PV Production Winter', 'Solar PV Production Summer', 'Retail Firm', 'Default Bias'
    ]
    df = pd.DataFrame(consumption_data, columns=columns)
    return df


In [8]:
## Correcting merge function

import pandas as pd

def merge_function(household_data, price_emission_data):
    # Ensure the 'Retail Firm' columns in both DataFrames are of type string
    household_data['Retail Firm'] = household_data['Retail Firm'].astype(str)
    price_emission_data['Retail Firm'] = price_emission_data['Retail Firm'].astype(str)

    # Merge household data with price_emission_data based on 'Retail Firm' for current offers
    current_data = pd.merge(household_data, price_emission_data, on='Retail Firm')

    return current_data

# Usage example, ensure you have 'household_data' and 'price_emission_data' DataFrames ready to use:
# merged_df = merge_function(household_data, price_emission_data)
# print(merged_df)


In [9]:
## Household function to minimize costs

import pandas as pd
from scipy.optimize import minimize

def optimize_consumption(data, season):
    # Determine the preferred consumption column based on season
    if season == 'summer':
        preferred_key = 'Summer Preferred'
    else:
        preferred_key = 'Winter Preferred'

    # Prepare columns for results in the original DataFrame
    data['Current Optimal Consumption'] = pd.Series(dtype=object)
    data['Current Total Consumption'] = pd.Series(dtype=float)
    data['Current Cost'] = pd.Series(dtype=float)
    data['Benchmark Optimal Consumption'] = pd.Series(dtype=object)
    data['Benchmark Total Consumption'] = pd.Series(dtype=float)
    data['Benchmark Cost'] = pd.Series(dtype=float)

    # Iterate through each row in the dataset
    for index, row in data.iterrows():
        preferred_consumption = row[preferred_key]
        current_offer = row['Retail Price Offer']
        benchmark_offer = row['Benchmark Offer']
        emissions = row['Emission']
        financial_weight = row['Financial Weight']
        carbon_weight = row['Carbon Weight']
        retail_firm = row['Retail Firm']  # Extract the Retail Firm

        # Function to perform optimization
        def perform_optimization(price_offer):
            # Normalize price
            max_price = max(price_offer)
            normalized_price = [p / max_price for p in price_offer]

            # Normalize emissions
            max_emission = max(emissions)
            normalized_emission = [e / max_emission for e in emissions]

            # Define the objective function
            def objective(consumption):
                financial_cost = sum(p * c for p, c in zip(normalized_price, consumption))
                carbon_cost = sum(e * c for e, c in zip(normalized_emission, consumption))
                return financial_weight * financial_cost + carbon_weight * carbon_cost

            # Bounds for each period's consumption
            bounds = [(0.9 * pref, 1.2 * pref) for pref in preferred_consumption]

            # Initial guess (starting from the preferred consumption)
            initial_guess = preferred_consumption

            # Perform optimization
            return minimize(objective, initial_guess, bounds=bounds, method='SLSQP')

        # Optimize current scenario
        current_result = perform_optimization(current_offer)
        optimal_current_consumption = current_result.x
        total_current_consumption = sum(optimal_current_consumption)
        current_cost = sum(p * c for p, c in zip(current_offer, optimal_current_consumption))

        # Optimize benchmark scenario
        benchmark_result = perform_optimization(benchmark_offer)
        optimal_benchmark_consumption = benchmark_result.x
        total_benchmark_consumption = sum(optimal_benchmark_consumption)
        benchmark_cost = sum(p * c for p, c in zip(benchmark_offer, optimal_benchmark_consumption))

        # Add results to the original DataFrame
        data.at[index, 'Retail Firm'] = retail_firm
        data.at[index, 'Current Optimal Consumption'] = [round(num, 2) for num in optimal_current_consumption]
        data.at[index, 'Current Total Consumption'] = round(total_current_consumption, 2)
        data.at[index, 'Current Cost'] = round(current_cost, 2)
        data.at[index, 'Benchmark Optimal Consumption'] = [round(num, 2) for num in optimal_benchmark_consumption]
        data.at[index, 'Benchmark Total Consumption'] = round(total_benchmark_consumption, 2)
        data.at[index, 'Benchmark Cost'] = round(benchmark_cost, 2)

    return data


In [10]:
def calculate_market_share(optimized_data):
    # Sum the 'Current Total Consumption' for each 'Retail Firm'
    total_consumption_by_firm = optimized_data.groupby('Retail Firm')['Current Total Consumption'].sum()

    # Calculate the total consumption in the market
    total_market_consumption = total_consumption_by_firm.sum()

    # Calculate the market share for each firm, rounded to three decimal places
    market_shares = (total_consumption_by_firm / total_market_consumption).round(3).reset_index()
    market_shares.columns = ['Retail Firm', 'Market Share']

    # Convert DataFrame to list of tuples
    market_share_tuples = list(market_shares.itertuples(index=False, name=None))

    return market_share_tuples


In [11]:
def switch_decision(data):
    # Define the columns to return
    columns_to_return = [
        'Household ID', 'Household Type','Default Bias', 'Base Consumption', 'Solar PV Capacity', 'Financial Weight', 'Carbon Weight',
        'Winter Gross', 'Summer Gross', 'Winter Preferred', 'Summer Preferred',
        'Solar PV Production Winter', 'Solar PV Production Summer', 'Retail Firm'
    ]

    # Iterate over each row in the DataFrame
    for index, row in data.iterrows():
        # Calculate if the Current Cost is more than 30% higher than the Benchmark Cost
        if row['Current Cost'] > ((1 + row['Default Bias']) * row['Benchmark Cost']):
            # Update the Retail Firm to the Benchmark Retail Firm
            data.at[index, 'Retail Firm'] = row['Benchmark Retail Firm']
        # If not, the Retail Firm remains the same (no action needed)

    # Return only the specified columns
    return data[columns_to_return]


**Application of these functions and model results**



 First Stage: Using generation value

In [12]:
## Upload power plant capacity from github and generate random values
import pandas as pd
import numpy as np

gen_url = 'https://raw.githubusercontent.com/smuratsirin/Retail_Market_Analysis/main/sa_power_plant_data.csv'
fossil_generator_data = pd.read_csv(gen_url)

# Columns to randomize
columns_to_randomize = ['Fixed_Costs', 'Variable_Costs', 'Emissions']
# Apply the function and set seed to 42 for replication
fossil_generator_data = randomize_costs_and_emissions(fossil_generator_data,
                                                      columns_to_randomize,
                                                      seed = 42)
print(fossil_generator_data.head())

      name  capacity type  fixed_costs  variable_costs  emissions
0  Plant 1    162.00  Gas    14.247241       27.731788   0.326524
1  Plant 2    263.40  Oil    29.507143       41.853300   0.725450
2  Plant 3    348.00  Oil    27.319939       49.132911   0.749718
3  Plant 4    532.70  Oil    25.986585       47.476034   0.564645
4  Plant 5     81.03  Oil    21.560186       51.969829   0.703386


In [13]:
## Now assign availability to these technologies
## By assigning availability this creates variation in the data
availability = 85 ## This value shoudl be percentage
generator_availability(fossil_generator_data, availability)
print(fossil_generator_data.head())


      name  capacity type  fixed_costs  variable_costs  emissions  \
0  Plant 1    162.00  Gas    14.247241       27.731788   0.326524   
1  Plant 2    263.40  Oil    29.507143       41.853300   0.725450   
2  Plant 3    348.00  Oil    27.319939       49.132911   0.749718   
3  Plant 4    532.70  Oil    25.986585       47.476034   0.564645   
4  Plant 5     81.03  Oil    21.560186       51.969829   0.703386   

   availability  
0             0  
1             1  
2             1  
3             1  
4             0  


In [14]:
## Upload demand profile and renewable energy data from Github
demand_url = 'https://raw.githubusercontent.com/smuratsirin/Retail_Market_Analysis/main/demand_and_renewable_data.csv'
demand_data = pd.read_csv(demand_url)
print(demand_data.head())


   Season  Month  peak_demand  \
0  Winter      1        35600   
1  Winter      2        35600   
2  Winter      3        40000   
3  Winter      4        50000   
4  Summer      5        55000   

                                      demand_profile installed_capacity  \
0  0.91,0.90,0.89,0.86,0.86,0.86,0.86,0.86,0.89,0...              0,0,0   
1  0.91, 0.90, 0.89, 0.86, 0.86, 0.86, 0.86, 0.86...              0,0,0   
2  0.91, 0.90, 0.89, 0.86, 0.86, 0.86, 0.86, 0.86...              0,0,0   
3  0.91, 0.90, 0.89, 0.86, 0.86, 0.86, 0.86, 0.86...              0,0,0   
4  0.92, 0.91, 0.91, 0.88, 0.88, 0.86, 0.86, 0.86...              0,0,0   

                           solar_capacity_factor_24h  \
0  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.18,0.41,0.58,0.6...   
1  0.0,0.0,0.0,0.0,0.0,0.0,0.03,0.25,0.46,0.62,0....   
2  0.0,0.0,0.0,0.0,0.0,0.0,0.06,0.28,0.47,0.61,0....   
3  0.0,0.0,0.0,0.0,0.0,0.0,0.07,0.24,0.41,0.55,0....   
4  0.0,0.0,0.0,0.0,0.0,0.01,0.08,0.23,0.39,0.52,0...   

             

In [18]:
# This function defines the generation_function
import pandas as pd
import numpy as np

period = 3
gen_row = demand_data.iloc[period]
peak_dem = gen_row['peak_demand']
demand_prof_24h = gen_row['demand_profile']
renewable_capacity = gen_row['installed_capacity']
solar_cap_fac_24h = gen_row['solar_capacity_factor_24h']
wind_cap_fac_24h = gen_row['wind_capacity_factor_24h']
renewable_costs = gen_row['costs']
voll = 75

prices, emissions = generation_function(pricing = "marginal",
                                        peak_demand = peak_dem,
                                        demand_profile_24h = demand_prof_24h,
                                        renewable_capacity = renewable_capacity,
                                        solar_capacity_factor_24h = solar_cap_fac_24h,
                                        wind_capacity_factor_24h = wind_cap_fac_24h,
                                        renewable_costs = renewable_costs,
                                        voll = voll,
                                        fossil_power_plants = fossil_generator_data)

print(prices)
print(emissions)


[4.3386892614641335, 4.3386892614641335, 4.177404116949125, 4.112712889879441, 4.112712889879441, 4.112712889879441, 4.112712889879441, 4.112712889879441, 4.177404116949125, 4.3386892614641335, 4.488832073455903, 4.488832073455903, 4.504822241811951, 4.504822241811951, 4.504822241811951, 4.504822241811951, 4.504822241811951, 4.504822241811951, 4.504822241811951, 4.504822241811951, 4.504822241811951, 4.504822241811951, 4.504822241811951, 4.488832073455903]
[0.5310776577437846, 0.5304666453299861, 0.5289327555385052, 0.5220046585802697, 0.5220046585802697, 0.5220046585802697, 0.5220046585802697, 0.5220046585802697, 0.5289327555385052, 0.5310776577437846, 0.5358954292823578, 0.5358954292823578, 0.5404640563311999, 0.5404640563311999, 0.5404640563311999, 0.5404640563311999, 0.5404640563311999, 0.5452722862233476, 0.5452722862233476, 0.5452722862233476, 0.5404640563311999, 0.5404640563311999, 0.5404640563311999, 0.5358954292823578]


In [ ]:
## This code creates generation results
import numpy as np
import pandas as pd

## Set a random seed for reproducibility of results
np.random.seed(42)
generation_info = pd.read_excel('/content/KAPSARC_Model_Generation.xlsx')


# Initialize a list to store results
generation_results = []

# Loop through 12 periods to simulate annual market behavior
for period in range(0, 12):
    print(f"Processing Period: {period}")

    gen_row = generation_info.iloc[period]
    gen_peak_dem = gen_row['peak_demand']
    gen_demand_prof_24h = gen_row['demand_profile']
    gen_capacity = gen_row['installed_capacity']
    solar_cap_fac_24h = gen_row['solar_capacity_factor_24h']
    wind_cap_fac_24h = gen_row['wind_capacity_factor_24h']
    gen_costs = gen_row['costs']
    gen_voll = gen_row['voll']

    ## Include generation function
    ## Simulate generation data (prices and emissions) for each hour
    ## Set Pricing
    prices, emissions = generation_function(pricing = "average",
                                            peak_demand = gen_peak_dem,
                                            demand_profile_24h = gen_demand_prof_24h,
                                            capacity = gen_capacity,
                                            solar_capacity_factor_24h = solar_cap_fac_24h,
                                            wind_capacity_factor_24h = wind_cap_fac_24h,
                                            costs = gen_costs,
                                            voll = gen_voll)

    generation_results.append([period, prices, emissions])

# Convert the list of results into a DataFrame
generation_results_df = pd.DataFrame(generation_results, columns=['Period', 'Prices', 'Emissions'])

# Print the results DataFrame
print(generation_results_df)


Processing Period: 0
Processing Period: 1
Processing Period: 2
Processing Period: 3
Processing Period: 4
Processing Period: 5
Processing Period: 6
Processing Period: 7
Processing Period: 8
Processing Period: 9
Processing Period: 10
Processing Period: 11
    Period                                             Prices  \
0        0  [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10....   
1        1  [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10....   
2        2  [9.97, 9.97, 9.97, 9.97, 9.97, 9.97, 9.97, 9.9...   
3        3  [9.626, 9.626, 9.626, 9.626, 9.626, 9.626, 9.6...   
4        4  [9.492, 9.492, 9.492, 9.492, 9.492, 9.492, 9.4...   
5        5  [9.263, 9.263, 9.263, 9.263, 9.263, 9.263, 9.2...   
6        6  [9.368, 9.368, 9.368, 9.368, 9.368, 9.368, 9.3...   
7        7  [9.263, 9.263, 9.263, 9.263, 9.263, 9.263, 9.2...   
8        8  [9.368, 9.368, 9.368, 9.368, 9.368, 9.368, 9.3...   
9        9  [9.642, 9.642, 9.642, 9.642, 9.642, 9.642, 9.6...   
10      10  [9.97, 9.97, 9.97, 

In [ ]:

import numpy as np
import pandas as pd

## Set a random seed for reproducibility of results
np.random.seed(42)

## Define time-related and retailer variables
hours = 24
time_periods = [(0, 4), (4, 8), (8, 12), (12, 16), (16, 20), (20, 24)]
retailers = [("Retail A", 0.34), ("Retail B", 0.34), ("Retail C", 0.32)]
gen_period = 3
gen_row = generation_results_df.iloc[gen_period]

prices =  gen_row['Prices']
emissions = gen_row['Emissions']

offer_df = retail_offer_function(retailers, prices, emissions, time_periods)
print(offer_df)

  Retail Firm                          Retail Price Offer  \
0    Retail A  [12.07, 12.07, 12.07, 12.07, 12.07, 12.07]   
1    Retail B  [12.07, 12.07, 12.07, 12.07, 12.07, 12.07]   
2    Retail C  [11.96, 11.96, 11.96, 11.96, 11.96, 11.96]   

                               Emission  \
0  [2.56, 2.56, 2.56, 2.56, 2.56, 2.56]   
1  [2.56, 2.56, 2.56, 2.56, 2.56, 2.56]   
2  [2.56, 2.56, 2.56, 2.56, 2.56, 2.56]   

                              Benchmark Offer Benchmark Retail Firm  
0  [11.96, 11.96, 11.96, 11.96, 11.96, 11.96]              Retail C  
1  [11.96, 11.96, 11.96, 11.96, 11.96, 11.96]              Retail C  
2  [11.96, 11.96, 11.96, 11.96, 11.96, 11.96]              Retail C  


In [ ]:
## Import necessary libraries
import numpy as np
import pandas as pd

## Set a random seed for reproducibility of results
np.random.seed(42)

## Define time-related and retailer variables
hours = 24
time_periods = [(0, 4), (4, 8), (8, 12), (12, 16), (16, 20), (20, 24)]
retailers = [("Retail A", 0.34), ("Retail B", 0.34), ("Retail C", 0.32)]

## Generate initial household dataset
household_df = create_household_dataset(N=100, seed=42, retailers_info=retailers)

# Loop through 12 periods to simulate annual market behavior
for period in range(1, 13):
    print(f"Processing Period: {period}")

    gen_period = period - 1
    gen_row = generation_results_df.iloc[gen_period]

    prices =  gen_row['Prices']
    emissions = gen_row['Emissions']

    ## Retrieve offer values from retailers based on the simulated data
    offer_df = retail_offer_function(retailers, prices, emissions, time_periods)

    ## Merge household data with offer data
    merged_df = merge_function(household_data=household_df, price_emission_data=offer_df)

    ## Determine the season based on the period number
    if period in [5, 6, 7, 8, 9, 10]:
        season = "summer"
    else:
        season = "winter"

    ## Run the optimization algorithm for the current period and season
    period_result = optimize_consumption(merged_df, season=season)
    period_result['Period'] = period
    period_result['Season'] = season

    ## Save the results to CSV for future analysis
    file_name = f"Output_Period_{period}.csv"
    period_result.to_csv(file_name, index=False)

    ## Calculate market share based on current period results and update for future use
    retailers = calculate_market_share(period_result)

    ## Update household data based on switching decisions
    household_df = switch_decision(period_result)

    print(f"Completed Period: {period}")

print("All periods processed and results saved.")



Processing Period: 1
Completed Period: 1
Processing Period: 2


<ipython-input-5-8986345cd16e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_data['Retail Firm'] = household_data['Retail Firm'].astype(str)


Completed Period: 2
Processing Period: 3


<ipython-input-5-8986345cd16e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_data['Retail Firm'] = household_data['Retail Firm'].astype(str)


Completed Period: 3
Processing Period: 4


...

Completed Period: 10
Processing Period: 11


<ipython-input-5-8986345cd16e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_data['Retail Firm'] = household_data['Retail Firm'].astype(str)


Completed Period: 11
Processing Period: 12


<ipython-input-5-8986345cd16e>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household_data['Retail Firm'] = household_data['Retail Firm'].astype(str)


Completed Period: 12
All periods processed and results saved.
